In [ ]:
from __future__ import print_function, division    # Python 2/3 compatibility
from skimage import io, color                             # utilities to read and write images in various formats
import numpy as np                                 # array manipulation package
import matplotlib.pyplot as plt                     # plotting package
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 15)
plt.rcParams['image.cmap'] = 'gray'                # set default colormap to gray

# Digital Image Processing - Programming Assignment \#3

**Please, fill in your personal details below.**

# Personal details:

* **Name(s) and student ID(s): XX** 
* **Contact information: XX** 

# 3. Image transforms - lowpass and highpass filtering in frequency domain

In the following, you will first perform ideal lowpass and highpass filtering on the test image, and later also Gaussian lowpass and highpass filtering are considered. First, read the part concerning image enhancement in frequency domain in the lecture notes or in the course book.

The deadline for returning your work is **April 14, 2021 at 23:59. Please, follow carefully the submission instructions given in the end of this notebook.** You are encouraged to seek information in other places than the course book and lecture material but remember **list all your sources under references**.

If you experience problems that you cannot solve using the course material or the Python documentation, or have any questions regarding to the programming assignments, please do not hesitate to contact the course assistant by e-mail at the address XX


**3.1. Read the test image `test.png`, convert it to grayscale and then display it.**

Hint: One way to convert the test image is to use [skimage.color module]( https://scikit-image.org/docs/dev/api/skimage.color.html) 

In [ ]:
# read test image
original = plt.imread("test.png")
# convert the image from rgb to gray
grayscale = color.rgb2gray(original)
# display the test image
plt.imshow(grayscale)
plt.show()

**3.2. Compute the Fourier transform (FT) of the test image and take a look at what the magnitude of the FT looks like.**

Hint: When plotting the FTs, use logarithmic graylevel transformation to make the result more illustrative for human visual system: 

`>>> np.log(np.abs(image_fft)+1)`

In [ ]:
from scipy import fftpack

# compute the FT of the test image using 'fftpack.fft2'
FT = fftpack.fft2(grayscale)
# translate the origin of the FT (low frequencies) to the center using 'ftpack.fftshift'
cFT = fftpack.fftshift(FT)
# display the magnitude of the uncentered and centered FT using 'imshow'.
plt.subplot(1,2,1)
plt.imshow(np.log(np.abs(FT)+1))
plt.subplot(1,2,2)
plt.imshow(np.log(np.abs(cFT)+1))
plt.show()

**The code for constructing a ideal lowpass filter is given below:**

In [ ]:
# make two frequency matrices, 'f1' and 'f2', as help variables (frequencies from -1 to 1)
n = (500,600)
f1 = ( np.arange(0,n[0])-np.floor(n[0]/2) ) * (2./(n[0]))
f2 = ( np.arange(0,n[1])-np.floor(n[1]/2) ) * (2./(n[1]))
f1, f2 = np.meshgrid(f2, f1)
# make a matrix with absolute values of frequency (“sampled” frequency domain)
D = np.sqrt(f1**2 + f2**2)

# set cut-off frequency D0 to 0.2
D0 = 0.2;

# filter matrix is initialized to ones 
Hlp = np.ones(n)


# set frequencies in filter mask Hlp greater than the cut-off frequency D0 to zero, other elements remain unaltered
Hlp[D>D0] = 0.0


**3.3. Modify the lowpass filter code and construct ideal highpass filter `Hhp` with the same cut-off frequency `D0=0.2` and display both ideal lowpass and highpass filter masks in the same figure.**

In [ ]:
# create ideal highpass filter mask Hhp
Hhp = np.zeros(n)
Hhp[D>D0] = 1
# display the filters
plt.subplot(1,2,1)
plt.imshow(Hlp)
plt.subplot(1,2,2)
plt.imshow(Hhp)
plt.show()

**3.4. Perform ideal lowpass and highpass filtering in the frequency domain by multiplying the centralized FT of the original image with the `Hlp` and `Hhp` filter masks (element-per-element matrix multiplication) and display the two resulting FTs in the same figure.**

In [ ]:
# apply ideal lowpass and highpass filtering to the test image, i.e. multiply element-wise the fft of the image with the filter masks
FTlow = np.multiply(cFT, Hlp)
FThigh = np.multiply(cFT, Hhp)
# display the magnitude of the resulting FTs
plt.subplot(1,2,1)
plt.imshow(np.log(np.abs(FTlow)+1))
plt.subplot(1,2,2)
plt.imshow(np.log(np.abs(FThigh)+1))
plt.show()

**3.5. Reconstruct the filtered images with `fftpack.ifft2()` and `fftpack.ifftshift()` in reverse order and display the two filtered images using `imshow()` in the same figure.** 

Hint: Due to round-off errors, you have to take the real part of the result of inverse FT before displaying it with `imshow()`. Please note also that the resulting images values beyond the original `uint8` image `[0,255]`, so you need to clip these values using `np.clip()`.

In [ ]:
# reconstruct the filtered images
rFTlow = fftpack.ifft2(fftpack.ifftshift(FTlow))
rFThigh = fftpack.ifft2(fftpack.ifftshift(FThigh))
# take the 'real' part of the resulting images due to possible round-off errors
reallow = np.real(rFTlow)
realhigh = np.real(rFThigh)
# clip values beyond the uint8 range [0,255] 
cliplow = np.clip(reallow, 0, 255)
cliphigh = np.clip(realhigh, 0, 255)
# display the original image and its lowpass and highpass filtered images in the same figure
plt.subplot(1,3,1)
plt.imshow(original)
plt.subplot(1,3,2)
plt.imshow(cliplow)
plt.subplot(1,3,3)
plt.imshow(cliphigh)
plt.show()

When performing ideal lowpass and highpass filtering, unwanted artefacts appear to the filtered image. **What is this phenomenon called and why does it occur?**

`Gibbs phenomenon, which is caused by the subtotal of the fourier series oscillating at the discontinuities of the filter function`

**3.6. Now, construct Gaussian lowpass and highpass filters with cut-off frequency `D0=0.2` and display them in the same figure.**

Hint: All you need to do is to modify the filter matrix `Hlp` line in the example code snippet accordingly to form `Hlpg` and `Hhpg` (see, formula 4.3-7 course book or lecture notes).

In [ ]:
# construct Gaussian lowpass and highpass filters
Hlpg = np.exp(-D**2/2/D0**2)
Hhpg = 1 - Hlpg
# display the filter masks
plt.subplot(1,2,1)
plt.imshow(Hlpg)
plt.subplot(1,2,2)
plt.imshow(Hhpg)
plt.show()

**3.7. Perform Gaussian lowpass and highpass filtering to the original test image and display the magnitude of the resulting FTs in the same figure.**

In [ ]:
# apply gaussian lowpass and highpass filtering to the test image
FTlowg = np.multiply(cFT, Hlpg)
FThighg = np.multiply(cFT, Hhpg)
# display the magnitude of the resulting FTs
plt.subplot(1,2,1)
plt.imshow(np.log(np.abs(FTlowg)+1))
plt.subplot(1,2,2)
plt.imshow(np.log(np.abs(FThighg)+1))
plt.show()

**3.8. Finally, reconstruct the filtered images like in step 4.5. and display the original image and the two Gaussian filtered images in the same figure.**

In [ ]:
# reconstruct the filtered images 
rFTlowg = fftpack.ifft2(fftpack.ifftshift(FTlowg)).real.clip(0,255)
rFThighg = fftpack.ifft2(fftpack.ifftshift(FThighg)).real.clip(0,255)
# display the three images in the same figure
plt.subplot(1,3,1)
plt.imshow(original)
plt.subplot(1,3,2)
plt.imshow(rFTlowg)
plt.subplot(1,3,3)
plt.imshow(rFThighg)
plt.show()

**Do the unwanted artefacts appear in the Gaussian lowpass filtered image, why or why not?**

`Gaussian filters are non-negative and non-oscillatory, and therefore cause no ringing.`

**What kind of effect does Gaussian (and ideal) lowpass filtering have on images in general? Why? What about highpass filtering? Why?**

`Lowpass filtering causes blurring on the image. This is because it filters out high-frequency information, in this case, edges. Highpass filtering causes sharpening on the image. This is because it increases the contrast between areas, for example edges.`

# Aftermath
Finally, fill your answers to the following questions:

**How much time did you need to complete this exercise?**

`2 hours.`

**Did you experience any problems with the exercise? Was there enough help available? Should this notebook be more (or less) detailed?**

`REPLACE THIS TEXT WITH YOUR ANSWER.`

# References
`Numpy, skimage and matplotlib documentation. Digital image processing, Gibbs phenomenon and ringing artifacts wikipedia pages.`

# Submission

1. Before submitting your work, **check that your notebook (code) runs from scratch** and reproduces all the requested results by clicking on the menu `Kernel -> Restart & Run All`! Also, check that you have answered all the questions written in **bold**.
2. Clear all outputs and variables, etc. by click on the menu `Kernel -> Restart & Clear Output`. This may (or will) reduce the file size of your deliverable a lot! 
3. Rename this Jupyter notebook to **`DIP_PA3_[student number(s)].ipynb`** (e.g. `DIP_PA3_1234567.ipynb` if solo work or `DIP_PA3_1234567-7654321.ipynb` if pair work)
4. Submit **only** the resulting Jupyter notebook (the file with extension `.ipynb`). Please **do not include your working folder or the test images** in your submission!